#📘 GET TO KNOW A DATASET: ivrit-ai Hebrew Audio v2


## 🧩 1. Introduction

This notebook provides a practical introduction to the ivrit-ai Hebrew Audio v2 dataset.
It shows how to explore the dataset when stored in a local directory, how to inspect sources, compute basic statistics, and listen to individual recordings.

The dataset is a large, curated collection of Hebrew audio files gathered from diverse public sources (podcasts, YouTube channels, radio archives, and other institutional material).
It is distributed via [Hugging Face](https://huggingface.co/datasets/ivrit-ai/audio-v2)

## 📦 2. Folder Structure Assumed in This Notebook
We assume the dataset is stored locally in a directory with the following structure:

```
data_root/
    source_A/
        file1.mp3
        file2.wav
        ...
    source_B/
        clip3.m4a
        clip4.opus
        ...
    ...

```


*   Each source is placed in its own folder.

*   Each folder contains raw audio files in arbitrary formats (.mp3, .m4a, .opus, .wav, etc.)
* No additional metadata is required for the tasks below.

Set your dataset path here:



In [ ]:
DATA_ROOT = "/path/to/ivrit-ai-audio-v2"


## 🔧 3. Setup

In [ ]:
import os
from pathlib import Path
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Audio, display


## 📁 4. Enumerate Sources and Audio Files

In [ ]:
DATA_ROOT = Path("/path/to/ivrit-ai-audio-v2")

sources = [p for p in DATA_ROOT.iterdir() if p.is_dir()]
print("Number of sources:", len(sources))
sources[:10]


## 🔢 5. Count Audio Files per Source

In [ ]:
audio_exts = {".mp3", ".wav", ".m4a", ".opus", ".flac"}

def list_audio_files(folder):
    return [f for f in folder.iterdir() if f.suffix.lower() in audio_exts]

records = []
for source in sources:
    files = list_audio_files(source)
    records.append({
        "source": source.name,
        "num_files": len(files)
    })

df_counts = pd.DataFrame(records).sort_values("num_files", ascending=False)
df_counts.head()


## ⏱️ 6. Calculate Total Duration per Source

This loads only metadata from audio headers — not full audio into memory.

In [ ]:
def get_duration(path):
    try:
        return librosa.get_duration(path=path)
    except Exception:
        return np.nan

duration_records = []

for source in sources:
    files = list_audio_files(source)
    for f in files:
        duration_records.append({
            "source": source.name,
            "file": f.name,
            "path": f,
            "duration_s": get_duration(f)
        })

df = pd.DataFrame(duration_records)
df.head()


### Group by Source

In [ ]:
df_source = (
    df.groupby("source")["duration_s"]
      .agg(["count", "sum", "mean"])
      .rename(columns={"count": "num_files", "sum": "total_hours", "mean": "avg_sec"})
)

df_source["total_hours"] = df_source["total_hours"] / 3600
df_source.head()


### Total dataset duration

In [ ]:
total_hours = df["duration_s"].sum() / 3600
total_hours


## ▶️ 7. Play a Specific Audio File

In [ ]:
example_source = df.iloc[0]["source"]
example_path = df.iloc[0]["path"]

print("Source:", example_source)
print("File:", example_path.name)

display(Audio(str(example_path)))


## 📊 8. Plot a Waveform + Spectrogram

In [ ]:
y, sr = librosa.load(example_path, sr=None)

plt.figure(figsize=(12, 4))
plt.title("Waveform")
plt.plot(y)
plt.show()

plt.figure(figsize=(12, 4))
plt.title("Spectrogram")
S = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
librosa.display.specshow(S, sr=sr, x_axis="time", y_axis="hz")
plt.colorbar()
plt.show()


## 📘 9. Summary Table

In [ ]:
summary = {
    "num_sources": len(df_source),
    "num_audio_files": len(df),
    "total_hours": total_hours
}
summary


## License
This dataset is released under the ivrit.ai license, a modified CC-BY license permitting use for machine-learning model training while prohibiting deepfake generation and certain misuse scenarios.
Full terms:
https://www.ivrit.ai/en/license-faqs/

## Citation
If you use this dataset, cite:

> Marmor, Yanir; Lifshitz, Yair; Snapir, Yoad; Misgav, Kinneret (2025). *Building an Accurate Open-Source Hebrew ASR System through Crowdsourcing*. Interspeech 2025.

